In [2]:
import numpy as np
import pandas as pd
import random
import operator
from math import e
from math import pi
from sklearn.model_selection import KFold

In [14]:
spectf=pd.read_csv('SPECTF_test.csv')
l=[i for i in range(0,44)]
data=spectf.iloc[:,l].values.tolist()
y=spectf.iloc[:,44].values.tolist()
print(y)

['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']


In [5]:
def group_class(data_list,x):
    
    group={}
    group['Yes']=[]
    group['No']=[]
    
    for i in data_list:
        if(y[i]=='Yes'):
            group['Yes'].append(data[i])
        elif(y[i]=='No'):
            group['No'].append(data[i])
       
    return group

In [6]:
def normal_pdf(x, mean, stdev):
        variance = stdev ** 2
        exp_squared_diff = (x - mean) ** 2
        exp_power = -exp_squared_diff / (2 * variance)
        exponent = e ** exp_power
        denominator = ((2 * pi) ** .5) * stdev
        normal_prob = exponent / denominator
        return normal_prob

In [7]:
def mean_data(data):
    return np.mean(data)

In [8]:
def std_data(data):
    return np.std(data)

In [9]:
def class_prob(group,target,data):
    return len(group[target])/(data)
    

In [10]:
def fun(features):
    for feature in zip(*features):
            yield {
                'stdev': std_data(feature),
                'mean': mean_data(feature)
            }

In [11]:
def joint_prob(row,summaries):
        joint_probs = {}
        for target, features in summaries.items():
            total_features = len(features['summary'])
            likelihood = 1
            for index in range(total_features):
                feature = row[index]
                mean = features['summary'][index]['mean']
                stdev = features['summary'][index]['stdev']
                normal_prob = normal_pdf(feature, mean, stdev)
                likelihood *= normal_prob
            prob = features['prob']
            joint_probs[target] =prob * likelihood
        return joint_probs

In [17]:
def predict(test,summaries):
    classes=[]
    for i in test:
        row=data[i]

        joint_probs=joint_prob(row,summaries)
        marginal_prob=sum(joint_probs.values())
        post_prob={}
#         print(joint_probs)
#         print(marginal_prob)
        for target,prob in joint_probs.items():
            post_prob[target]=prob/marginal_prob
#         print(post_prob)
        x=max(post_prob,key=post_prob.get)
       
        classes.append(x)
    return classes

In [18]:
def accuracy(classes,test_list,x):
    corr=0
    n=0
    test_list=test_list.tolist()

    for i in test_list:
       
        if(y[i]==classes[n]):
            corr+=1
        n+=1
    return corr/len(test_list)

In [46]:
def confusion_mat(classes,test_index,x):
   
    rows,cols=(2,2)
    n=0
    cm=[[0 for i in range(cols)] for j in range(rows)]
    for i in test_index:
        if(y[i]=='Yes'):
            if(classes[n]=='Yes'):
                cm[0][0]+=1
            elif(classes[n]=='No'):
                cm[0][1]+=1
            
        elif(y[i]=='No'):
            if(classes[n]=='Yes'):
                cm[1][0]+=1
            elif(classes[n]=='No'):
                cm[1][1]+=1
           
        n+=1
        
    print("  Confusion Matrix")
    print(cm)
    if(cm[0][0]+cm[0][1]==0):
        if(cm[0][0]==0 and cm[0][1]==0 and cm[1][0]==0):
            print("  Sensitivity is {}".format(1.0))
        else:
            print("  Sensitivity is {}".format(0.0))
    else:
        
        sensitivity=cm[0][0]/(cm[0][0]+cm[0][1])
        print("  Sensitivity is {}".format(sensitivity))
    if(cm[1][1]+cm[1][0]==0):
        if(cm[0][1]==0):
              print("  Specificity is {}".format(1.0))
        else:
             print("  Specificity is {}".format(0.0))
    else:
        spec=cm[1][1]/(cm[1][1]+cm[1][0])
        print("  Specificity is {}".format(spec))
        
            

In [47]:
kf=KFold(n_splits=10, shuffle=False)
acc_mean=[]
x=0
for train_index, test_index in kf.split(spectf):

    group=group_class(train_index,x)

    summaries = {}
    for target,features in group.items():
        summaries[target]={
            'prob':class_prob(group, target, len(train_index)),
            'summary': [i for i in fun(features)],
        }
    

    test=group_class(test_index,x)
    classes=predict(test_index,summaries)
    
        
   
    print("{} fold States------------->".format(x+1))
    acc=accuracy(classes,test_index,x)
    acc_mean.append(acc)
    confusion_mat(classes,test_index,x)
    
    print("  Accuracy is {}".format(acc*100))
    
    x+=1
print("\n")
    
print("      Average Accuracy of the model is {}".format(np.mean(acc_mean)*100))

1 fold States------------->
  Confusion Matrix
[[10, 1], [0, 0]]
  Sensitivity is 0.9090909090909091
  Specificity is 0.0
  Accuracy is 90.9090909090909
2 fold States------------->
  Confusion Matrix
[[1, 3], [2, 5]]
  Sensitivity is 0.25
  Specificity is 0.7142857142857143
  Accuracy is 54.54545454545454
3 fold States------------->
  Confusion Matrix
[[1, 2], [1, 7]]
  Sensitivity is 0.3333333333333333
  Specificity is 0.875
  Accuracy is 72.72727272727273
4 fold States------------->
  Confusion Matrix
[[3, 8], [0, 0]]
  Sensitivity is 0.2727272727272727
  Specificity is 0.0
  Accuracy is 27.27272727272727
5 fold States------------->
  Confusion Matrix
[[7, 4], [0, 0]]
  Sensitivity is 0.6363636363636364
  Specificity is 0.0
  Accuracy is 63.63636363636363
6 fold States------------->
  Confusion Matrix
[[7, 4], [0, 0]]
  Sensitivity is 0.6363636363636364
  Specificity is 0.0
  Accuracy is 63.63636363636363
7 fold States------------->
  Confusion Matrix
[[4, 0], [0, 7]]
  Sensitivity i